# Создание нейросети с помощью NumPy

В этом ноутбуке мы:
1. Реализуем простой нейрон используя NumPy
2. Вычислим производные для обратного распространения
3. Создадим класс слоя, который сохраняет состояния необходимые для прямого и обратного прохода
4. Создадим нейросеть из нескольких слоев
5. Обучим сеть на простой задаче


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)


## 1. Реализация простого нейрона

Нейрон выполняет следующую операцию:
$$y = \sigma(w^T x + b)$$

где:
- $x$ - входной вектор
- $w$ - вектор весов
- $b$ - смещение
- $\sigma$ - функция активации (будем использовать сигмоиду)


In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def neuron_forward(x, w, b):
    z = np.dot(w, x) + b
    output = sigmoid(z)
    return output

x = np.array([1.0, 2.0, 3.0])
w = np.array([0.5, -0.2, 0.1])
b = 0.3

output = neuron_forward(x, w, b)
print(f"Input: {x}")
print(f"Weights: {w}")
print(f"Bias: {b}")
print(f"Output: {output:.4f}")


## 2. Вычисление производных для обратного распространения

Для обратного распространения нам нужно вычислить градиенты:

**Производная сигмоиды:**
$$\frac{\partial \sigma}{\partial z} = \sigma(z) \cdot (1 - \sigma(z))$$

**Градиенты по параметрам:**
- $\frac{\partial L}{\partial w} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial z} \cdot \frac{\partial z}{\partial w} = \frac{\partial L}{\partial y} \cdot \sigma'(z) \cdot x$
- $\frac{\partial L}{\partial b} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial z} \cdot \frac{\partial z}{\partial b} = \frac{\partial L}{\partial y} \cdot \sigma'(z)$


In [ ]:
def sigmoid_derivative(z):
    s = sigmoid(z)
    return s * (1 - s)

def neuron_backward(x, w, b, dy):
    z = np.dot(w, x) + b
    dz = dy * sigmoid_derivative(z)
    dw = dz * x
    db = dz
    dx = dz * w
    return dw, db, dx

x = np.array([1.0, 2.0, 3.0])
w = np.array([0.5, -0.2, 0.1])
b = 0.3
dy = 1.0

dw, db, dx = neuron_backward(x, w, b, dy)
print(f"Gradient w.r.t. weights: {dw}")
print(f"Gradient w.r.t. bias: {db:.4f}")
print(f"Gradient w.r.t. input: {dx}")


## 3. Класс слоя нейронов с управлением состоянием

Теперь создадим класс слоя, который:
- Содержит несколько нейронов (принимает n_features входов и возвращает n_outputs выходов)
- Хранит параметры (матрицу весов и вектор смещений)
- Кэширует промежуточные значения во время прямого прохода
- Использует кэшированные значения для эффективного обратного прохода
- Реализует обновление параметров


In [ ]:
class Layer:
    def __init__(self, n_inputs, n_outputs):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.W = np.random.randn(n_inputs, n_outputs) * 0.01
        self.b = np.zeros(n_outputs)
        self.cache = {}
        
    def forward(self, x):
        z = np.dot(x, self.W) + self.b
        output = sigmoid(z)
        self.cache['x'] = x
        self.cache['z'] = z
        self.cache['output'] = output
        return output
    
    def backward(self, dy):
        x = self.cache['x']
        output = self.cache['output']
        dz = dy * output * (1 - output)
        
        if x.ndim == 1:
            self.dW = np.outer(x, dz)
            self.db = dz
            dx = np.dot(dz, self.W.T)
        else:
            batch_size = x.shape[0]
            self.dW = np.dot(x.T, dz) / batch_size
            self.db = np.mean(dz, axis=0)
            dx = np.dot(dz, self.W.T)
        
        return dx
    
    def update_parameters(self, learning_rate):
        self.W -= learning_rate * self.dW
        self.b -= learning_rate * self.db
    
    def get_parameters(self):
        return {'W': self.W.copy(), 'b': self.b.copy()}


## 4. Пример использования класса


In [ ]:
layer = Layer(n_inputs=3, n_outputs=2)

x = np.array([1.0, 2.0, 3.0])
output = layer.forward(x)
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Output: {output}")

dy = np.ones(2)
dx = layer.backward(dy)
print(f"Градиент по входу: {dx}")

layer.update_parameters(learning_rate=0.1)
params = layer.get_parameters()
print(f"Размер матрицы весов: {params['W'].shape}")
print(f"Размер вектора смещений: {params['b'].shape}")


## 5. Создание нейросети из нескольких слоев

Теперь создадим класс нейросети, которая будет:
- Состоять из нескольких слоев
- Последовательно выполнять прямой проход через все слои
- Последовательно выполнять обратный проход через все слои (в обратном порядке)
- Обновлять параметры всех слоев


In [ ]:
class NeuralNetwork:
    def __init__(self, layer_sizes):
        self.layers = []
        for i in range(len(layer_sizes) - 1):
            layer = Layer(layer_sizes[i], layer_sizes[i + 1])
            self.layers.append(layer)
    
    def forward(self, x):
        output = x
        for layer in self.layers:
            output = layer.forward(output)
        return output
    
    def backward(self, dy):
        dx = dy
        for layer in reversed(self.layers):
            dx = layer.backward(dx)
        return dx
    
    def update_parameters(self, learning_rate):
        for layer in self.layers:
            layer.update_parameters(learning_rate)
    
    def get_all_parameters(self):
        params = []
        for i, layer in enumerate(self.layers):
            params.append({
                'layer': i,
                'parameters': layer.get_parameters()
            })
        return params


## 6. Пример использования нейросети


In [ ]:
network = NeuralNetwork([4, 8, 3, 1])

print("Архитектура сети:")
for i, layer in enumerate(network.layers):
    print(f"  Слой {i+1}: {layer.n_inputs} -> {layer.n_outputs}")

x = np.array([1.0, 2.0, 3.0, 4.0])
output = network.forward(x)
print(f"\nВход: {x}")
print(f"Выход сети: {output}")

dy = np.array([1.0])
dx = network.backward(dy)
print(f"\nГрадиент по входу: {dx}")

network.update_parameters(learning_rate=0.01)
print("\nПараметры обновлены!")


## 7. Обучение сети на простой задаче

Обучим сеть предсказывать простую функцию


In [ ]:
def mse_loss(y_pred, y_true):
    return np.mean((y_pred - y_true) ** 2)

def mse_loss_grad(y_pred, y_true):
    return 2 * (y_pred - y_true) / y_true.size

X_train = np.array([
    [0.0, 0.0],
    [0.0, 1.0],
    [1.0, 0.0],
    [1.0, 1.0]
])

y_train = np.array([[0.0], [1.0], [1.0], [0.0]])

network = NeuralNetwork([2, 4, 1])

losses = []
epochs = 1000
learning_rate = 0.5

for epoch in range(epochs):
    total_loss = 0
    for i in range(len(X_train)):
        x = X_train[i]
        y_true = y_train[i]
        
        y_pred = network.forward(x)
        loss = mse_loss(y_pred, y_true)
        total_loss += loss
        
        grad = mse_loss_grad(y_pred, y_true)
        network.backward(grad)
        network.update_parameters(learning_rate)
    
    avg_loss = total_loss / len(X_train)
    losses.append(avg_loss)
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {avg_loss:.4f}")

print("\nПредсказания после обучения:")
for i in range(len(X_train)):
    pred = network.forward(X_train[i])
    print(f"  {X_train[i]} -> {pred[0]:.4f} (истинное: {y_train[i][0]})")


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Кривая обучения')
plt.grid(True)
plt.show()
